# Packing a dimer on a cubic lattice with `lammpsio`
In this tutorial, we’ll walk through filling a box with dimers using `lammpsio`. We'll create a simple cubic lattice where each unit cell contains a bonded dimer made up of two particle of type A and B. At the end, we will create a data file ready to be used by `LAMMPS`. 

First, we import `numpy` to make setting up the simple cubic lattice and `lammpsio` to handle creating the data file.

In [29]:
import lammpsio

import numpy

## Creating the simple cubic lattice

We define the core parameters of our system, including particle diameters, offset between bonded particles, and the number of repetitions of the unit cell. We choose the particles to have a unit diameter $d$ and a bond spacing of 1.5 $d$. We chose a lattice spacing of `2 * diameter + bond_length` so that each dimer are at the equilibrium bond distance. We also choose to place 1000 dimers total (10 in each direction).  

In [30]:
diameter = 1.0
bond_length = 1.5
lattice_spacing = 2 * diameter + bond_length
num_repeat = [10, 10, 10]

### Define the unit cell

We'll make our lattice with a simple cubic unit cell. Each unit cell contains two particles: type A at the origin, and type B shifted along the x-axis to give the proper spacing.

In [31]:
unit_cell = numpy.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * lattice_spacing
unit_cell_coords = numpy.array([
    [0, 0, 0],
    [bond_length, 0, 0]
])
unit_cell_typeids = [1, 2]
unit_cell_mass = [1.0, 1.5]

### Define `lammpsio.Box`



`lammpsio.Box` defines the simulation box using LAMMPS's convention. In LAMMPS, the simulation box is specified by three parameters: `low`, `high`, and `tilt`. The `low` parameter defines the origin (lower corner) of the box, while `high` specifies how far the box extends along each axis. The `tilt` parameter contains tilt factors (xy, xz, yz) that skew the edges to create non-orthorhombic simulation boxes.

These parameters can be transformed into a box matrix with the following form:

$$
\begin{bmatrix}
    a_x & b_x & c_x \\
    0 & b_y & c_y \\
    0 & 0 & c_z
\end{bmatrix},
$$

where each column of this matrix represents one of the three edge vectors (**A**, **B**, & **C**) that define the triclinic simulation box, and each of the matrix elements are derived from the `low`, `high`, and `tilt` values. The diagonal elements ($a_x$, $b_y$, $c_z$) represent the box lengths along each axis, while the off-diagonal elements come from the tilt factors. For more details on how to convert between the LAMMPS parameters and box matrix see the [LAMMPS documentation](https://docs.lammps.org/Howto_triclinic.html#transformation-from-general-to-restricted-triclinic-boxes).

For our orthorhombic unit cell, all tilt factors are zero, so `tilt` has the default value of `None`, resulting in a simple rectangular box. We choose `low` to be at `[0, 0, 0]` and `high` is the diagonal values of the box. 

In [32]:
box_matrix = (unit_cell * num_repeat).T
low = [0, 0, 0]
high = [box_matrix[0, 0], box_matrix[1, 1], box_matrix[2, 2]]
box = lammpsio.Box(low=low, high=high, tilt=None)

### Generate particle type IDs and positions

We calculate the total number of particles by multiplying the number of unit cells by the number of particles per cell.

In [33]:
num_cells = num_repeat[0] * num_repeat[1] * num_repeat[2]
N = num_cells * len(unit_cell_coords)

## Create a `lammpsio.Snapshot`

The `Snapshot` holds the data about the particle's configuration and properties, the `Box`, and the timestep. 


In [34]:
snap = lammpsio.Snapshot(N=N, box=box)

Particle attributes are assigned directly using the Snapshot's attributes. We generate positions for all particles by iterating through each unit cell position in our 3D lattice grid. For each unit cell, we calculate the absolute coordinates by scaling the unit cell position by the lattice spacing and adding the unit cell's internal particle coordinates. The particles are then positioned within the simulation box starting from `snap.box.low`.

Note that lammpsio automatically allocates an array with the right data type and shape for attributes, so we assign positions directly to snap.position rather than using an intermediate array!

In [35]:
for i, pos in enumerate(numpy.ndindex(*num_repeat)):
    first = i * len(unit_cell_coords)
    last = first + len(unit_cell_coords)
    absolute_coords = numpy.array(pos) * lattice_spacing + unit_cell_coords
    snap.position[first:last] = absolute_coords + snap.box.low

We then create an array of type IDs by replicating our unit cell pattern (typeID 1 and 2) across all cells. We do the same thing to give the particles their mass. 


In [36]:
snap.typeid = numpy.tile(unit_cell_typeids, num_cells)
snap.mass = numpy.tile(unit_cell_mass, num_cells)

To specify bonds, we assign the `Snapshot` a `lammpsio.Bonds` object. `lammpsio.Bonds` holds information about the bond typeIDs and which particles are members of the bond.

We know that each cell contains one dimer and thus one bond. Since all of our bonds in this system are the same, we assign them all typeID 1. 

In [37]:
N_bonds = num_cells

snap.bonds = lammpsio.Bonds(N=N_bonds, num_types=1)
snap.bonds.typeid = numpy.ones(N_bonds)

Each dimer consists of consecutive particle IDs (1-2, 3-4, etc.). We create bonds by connecting these consecutive pairs, resulting in 1000 bonds for our 1000 dimers.

In [38]:
snap.bonds.members = [[2 * i + 1, 2 * i + 2] for i in range(N_bonds)]

### Save to LAMMPS

Finally, we save the configuration to a data file ready to be used in `LAMMPS`. The data file is written in the molecular style since we have bonds. 

In [39]:
lammpsio.DataFile.create(filename="dimer_lattice.data", snapshot=snap, atom_style="molecular")

### Save to HOOMD-blue's GSD format

HOOMD-blue is molecular dynamics engine commonly used to study soft matter. There may come a time when you want to use HOOMD-blue or share your LAMMPS data file with someone who is more familar with it. Converting the data file to HOOMD-blue's GSD format manually can be tedious, but luckily `lammpsio` allows for quick and seamless conversion between the formats! 

Since HOOMD-blue requires alphanumeric types along with TypeIDs we have to add those to our snapshot. The `lammpsio.LabelMap` is effectively a dictionary associating a label (type) with a particle's or connection's typeid. Here we're going to assign the `Snapshot` a type_label mapping particle typeID `1 -> A` and `2 -> B` and bond typeid `1 -> dimer`. `lammpsio` will use the `LabelMap` to set the alphanumeric types in the GSD file. 

Note: LAMMPS now also supports alphanumeric type labeling, though `lammpsio` does not currently support for this feature, it is planned as a future addition. 

In [40]:
snap.type_label = lammpsio.LabelMap({1: "A", 2: "B"})
snap.bonds.type_label = lammpsio.LabelMap({1: "dimer"})

Since HOOMD-blue requires the simulation box to be centered about the origin, we need to shift our box coordinates. Currently, our box extends from `[0, 0, 0]` to `[35, 35, 35]`, but HOOMD-blue expects it to be centered at zero, ranging from `[-17.5, -17.5, -17.5]` to `[17.5, 17.5, 17.5]`. We accomplish this by calculating the extent of the box (`high - low`) and then shifting both the high and low boundaries by subtracting half the extent. We then do the same thing to `snap.position`. 

In [41]:
extent = snap.box.high - snap.box.low

snap.box.high -= extent / 2
snap.box.low -= extent / 2

snap.position -= extent / 2

Using `Snapshot.to_hoomd_gsd()` returns a HOOMD-blue GSD frame object. We can write this out to file and have the same particle and bond data ready to be used in a different simulation engine! You can also use `Snapshot.from_hoomd_gsd` to convert a HOOMD-blue GSD frame into a `Snapshot`! 

In [42]:
import gsd.hoomd
snap_gsd = snap.to_hoomd_gsd()

with gsd.hoomd.open("dimer_lattice.gsd", "w") as f:
    f.append(snap_gsd)
    f.flush()
    f.close()